<a href="https://colab.research.google.com/github/TorexAE/FundProg_202510/blob/main/Programa_Proyecto_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmo,Codigo y pseudocodigo del proyecto regulador de temperatura

##ALGORITMO


1. Iniciar el programa.
2. Conectarse a la red Wi-Fi usando SSID y contraseña.

   * Activar el módulo Wi-Fi.
   * Esperar hasta que esté conectado.
   * Mostrar la IP asignada.
3. Encender LED de energía.
4. Encender LED verde 1 para indicar que el sistema está listo.
5. Iniciar el servidor web en un hilo separado:

   * Escuchar requests HTTP en el puerto 80.
   * Si se recibe `/on`, habilitar el sistema y apagar LEDs rojos.
   * Si se recibe `/off`, deshabilitar el sistema.
   * Leer temperatura del sensor y mostrar la página web con los datos actuales.
6. Iniciar el bucle principal del sistema:

   * Si el sistema está en espera, esperar 1 segundo y continuar.
   * Si el sistema está deshabilitado:

     * Apagar motor, buzzer, LED amarillo y LED verde 2.
     * Esperar 1 segundo y continuar.
   * Leer temperatura del sensor DHT11.
   * Si ocurre un error en la lectura, esperar 2 segundos y volver a intentar.
7. Evaluar la temperatura medida:

   * Si la temperatura supera el umbral (ej. 30 °C):

     * Encender motor (in1 ON, in2 OFF).
     * Encender LED amarillo y apagar LED verde 2.
     * Si es la primera vez que supera el umbral, guardar el tiempo de inicio.
     * Si han pasado más de 25 segundos desde el inicio sin bajar temperatura:

       * Apagar motor y LED amarillo.
       * Encender LEDs rojos.
       * Activar buzzer en patrón intermitente.
       * Marcar el tiempo de inicio del LED rojo.
       * Poner sistema en estado de espera.
   * Si la temperatura está por debajo del umbral:

     * Apagar motor y buzzer.
     * Apagar LED amarillo y encender LED verde 2.
     * Reiniciar el contador de tiempo.
8. Apagar LEDs rojos después de 10 segundos si estaban encendidos.
9. Esperar 1 segundo y repetir el ciclo.



##PSEUDOCODIGO


```
Program programa_1

Start

    DECLARE boolean sistema_habilitado ← TRUE
    DECLARE boolean led_rojo_encendido ← FALSE
    DECLARE boolean estado_espera ← FALSE
    DECLARE real temp_umbral ← 30
    DECLARE real tiempo_max_regulacion ← 25
    DECLARE real tiempo_led_rojo ← 10

    CALL connect_wifi()
    CALL start_web_server_thread()
    CALL main_loop()

End

Function connect_wifi()
    ENABLE Wi-Fi in client mode
    CONNECT to network using SSID 'TOMAS-LAP' and password '9C2<n464'
    WHILE not connected DO
        WAIT 1 second
    END WHILE
    DISPLAY assigned IP
Return

Function start_web_server_thread()
    START thread to run web_server()
Return

Function web_server()
    CREATE socket on IP 0.0.0.0 and port 80
    LISTEN for incoming connections

    WHILE TRUE DO
        WAIT for client
        READ HTTP request

        IF request CONTAINS '/on' THEN
            SET sistema_habilitado ← TRUE
            SET led_rojo_encendido ← FALSE
            TURN OFF led_rojo_1 and led_rojo_2
            SET estado_espera ← FALSE
        ELSE IF request CONTAINS '/off' THEN
            SET sistema_habilitado ← FALSE
        END IF

        TRY to read DHT11 sensor
        IF read SUCCESS THEN
            DECLARE real temp
            SET temp ← measured temperature
        ELSE
            SET temp ← "--"
        END IF

        BUILD HTML page with temperature and system status
        SEND HTTP response to client
        CLOSE connection
    END WHILE
Return

Function sonar_buzzer()
    FOR 2 cycles DO
        FOR 3 pulses DO
            ACTIVATE buzzer at 1000 Hz
            WAIT 0.3 seconds
            DEACTIVATE buzzer
            WAIT 0.7 seconds
        END FOR
    END FOR
Return

Function main_loop()
    DECLARE real tiempo_inicio ← 0
    DECLARE real tiempo_inicio_led_rojo ← 0
    DECLARE real temp

    SET led_energia ← ON
    SET led_verde_1 ← ON

    WHILE TRUE DO
        IF estado_espera = TRUE THEN
            WAIT 1 second
            CONTINUE
        END IF

        IF sistema_habilitado = FALSE THEN
            TURN OFF motor and leds
            DEACTIVATE buzzer
            WAIT 1 second
            CONTINUE
        END IF

        TRY to read DHT11 sensor
        IF read SUCCESS THEN
            SET temp ← measured temperature
        ELSE
            DISPLAY error
            WAIT 2 seconds
            CONTINUE
        END IF

        IF temp > temp_umbral THEN
            TURN ON motor (in1 ON, in2 OFF)
            TURN ON led_amarillo, TURN OFF led_verde_2
            IF tiempo_inicio = 0 THEN
                SET tiempo_inicio ← current time
            END IF

            IF time elapsed > tiempo_max_regulacion THEN
                DISPLAY overheat alert
                TURN ON red leds, TURN OFF motor and led_amarillo
                CALL sonar_buzzer()
                SET tiempo_inicio ← 0
                SET led_rojo_encendido ← TRUE
                SET tiempo_inicio_led_rojo ← current time
                SET estado_espera ← TRUE
            END IF
        ELSE
            TURN OFF motor and buzzer
            TURN OFF led_amarillo, TURN ON led_verde_2
            SET tiempo_inicio ← 0
        END IF

        IF led_rojo_encendido = TRUE AND time elapsed since tiempo_inicio_led_rojo ≥ tiempo_led_rojo THEN
            TURN OFF red leds
            SET led_rojo_encendido ← FALSE
        END IF

        WAIT 1 second
    END WHILE
Return



```

##CODIGO

In [ ]:
import network
import socket
import time
import dht
from machine import Pin, PWM
import _thread


sensor = dht.DHT11(Pin(13))
motor_in1 = Pin(9, Pin.OUT)
motor_in2 = Pin(10, Pin.OUT)
buzzer = PWM(Pin(6))
buzzer.deinit()


led_verde_1 = Pin(0, Pin.OUT)
led_verde_2 = Pin(1, Pin.OUT)
led_amarillo = Pin(2, Pin.OUT)
led_rojo_1 = Pin(3, Pin.OUT)
led_rojo_2 = Pin(4, Pin.OUT)
led_energia = Pin(8, Pin.OUT)


temp_umbral = 30
tiempo_max_regulacion = 25
tiempo_led_rojo = 10
sistema_habilitado = True
led_rojo_encendido = False
tiempo_inicio_led_rojo = 0
estado_espera = False


def connect_wifi():
    wlan = network.WLAN(network.STA_IF)
    wlan.active(True)
    wlan.connect('TOMAS-LAP', '9C2<n464')
    print("Conectando a Wi-Fi...")
    while not wlan.isconnected():
        time.sleep(1)
    print('Conectado a Wi-Fi:', wlan.ifconfig())


def web_server():
    global sistema_habilitado, led_rojo_encendido, estado_espera

    addr = socket.getaddrinfo('0.0.0.0', 80)[0][-1]
    s = socket.socket()
    s.bind(addr)
    s.listen(1)
    print('Servidor web iniciado en', addr)

    while True:
        cl, addr = s.accept()
        request = cl.recv(1024).decode()
        print('Solicitud de', addr, request)


        if '/on' in request:
            sistema_habilitado = True
            led_rojo_encendido = False
            led_rojo_1.off()
            led_rojo_2.off()
            estado_espera = False
            print("Sistema HABILITADO remotamente")
        elif '/off' in request:
            sistema_habilitado = False
            print("Sistema DESHABILITADO remotamente")


        try:
            sensor.measure()
            temp = sensor.temperature()
        except:
            temp = "--"

        estado = "HABILITADO" if sistema_habilitado else "DESHABILITADO"

        # Página HTML
        html = f"""<!DOCTYPE html>
<html>
<head>
    <meta charset="UTF-8">
    <title>Control de Temperatura</title>
    <style>
        body {{
            font-family: Arial, sans-serif;
            text-align: center;
            background-color: #f4f4f4;
            color: #333;
        }}
        h2 {{
            color: #4CAF50;
        }}
        button {{
            padding: 10px 20px;
            font-size: 16px;
            margin: 10px;
            background-color: #4CAF50;
            color: white;
            border: none;
            border-radius: 5px;
            cursor: pointer;
        }}
        button:hover {{
            background-color: #45a049;
        }}
        p {{
            font-size: 18px;
        }}
    </style>
</head>
<body>
    <h2>Control de Temperatura - Proyecto</h2>
    <p><strong>Temperatura actual:</strong> {temp}°C</p>
    <p><strong>Estado del sistema:</strong> {estado}</p>
    <form action="/on" method="get">
        <button>Activar</button>
    </form>
    <br>
    <form action="/off" method="get">
        <button>Desactivar</button>
    </form>
</body>
</html>"""

        cl.send("HTTP/1.1 200 OK\r\nContent-Type: text/html\r\n\r\n")
        cl.sendall(html.encode())
        cl.close()


def sonar_buzzer():
    for _ in range(2):
        for _ in range(3):
            buzzer.freq(1000)
            buzzer.duty_u16(32768)
            time.sleep(0.3)
            buzzer.duty_u16(0)
            time.sleep(0.7)


def main():
    global sistema_habilitado, led_rojo_encendido, tiempo_inicio_led_rojo, estado_espera
    connect_wifi()

    led_energia.value(1)
    led_verde_1.on()
    tiempo_inicio = 0

    while True:
        if estado_espera:

            time.sleep(1)
            continue

        if not sistema_habilitado:

            motor_in1.off()
            motor_in2.off()
            buzzer.deinit()
            led_amarillo.off()
            led_verde_2.off()
            time.sleep(1)
            continue

        try:
            sensor.measure()
            temp = sensor.temperature()
            print("Temperatura:", temp)
        except Exception as e:
            print("Error lectura sensor:", e)
            time.sleep(2)
            continue

        if temp > temp_umbral:
            motor_in1.on()
            motor_in2.off()
            led_amarillo.on()
            led_verde_2.off()

            if tiempo_inicio == 0:
                tiempo_inicio = time.time()

            if time.time() - tiempo_inicio > tiempo_max_regulacion:
                print("No se ha logrado regular la temperatura.")


                led_rojo_1.on()
                led_rojo_2.on()
                led_amarillo.off()


                motor_in1.off()
                motor_in2.off()


                sonar_buzzer()

                tiempo_inicio = 0
                led_rojo_encendido = True
                tiempo_inicio_led_rojo = time.time()
                estado_espera = True
        else:

            motor_in1.off()
            motor_in2.off()
            buzzer.deinit()
            led_amarillo.off()
            led_verde_2.on()
            tiempo_inicio = 0


        if led_rojo_encendido and (time.time() - tiempo_inicio_led_rojo >= tiempo_led_rojo):
            led_rojo_1.off()
            led_rojo_2.off()
            led_rojo_encendido = False

        time.sleep(1)


_thread.start_new_thread(web_server, ())
main()




import network
import socket
import time
import dht
from machine import Pin, PWM
import _thread


sensor = dht.DHT11(Pin(13))
motor_in1 = Pin(9, Pin.OUT)
motor_in2 = Pin(10, Pin.OUT)
buzzer = PWM(Pin(6))
buzzer.deinit()


led_verde_1 = Pin(0, Pin.OUT)
led_verde_2 = Pin(1, Pin.OUT)
led_amarillo = Pin(2, Pin.OUT)
led_rojo_1 = Pin(3, Pin.OUT)
led_rojo_2 = Pin(4, Pin.OUT)
led_energia = Pin(8, Pin.OUT)


temp_umbral = 30
tiempo_max_regulacion = 25
tiempo_led_rojo = 10
sistema_habilitado = True
led_rojo_encendido = False
tiempo_inicio_led_rojo = 0
estado_espera = False


def connect_wifi():
    wlan = network.WLAN(network.STA_IF)
    wlan.active(True)
    wlan.connect('TOMAS-LAP', '9C2<n464')
    print("Conectando a Wi-Fi...")
    while not wlan.isconnected():
        time.sleep(1)
    print('Conectado a Wi-Fi:', wlan.ifconfig())


def web_server():
    global sistema_habilitado, led_rojo_encendido, estado_espera

    addr = socket.getaddrinfo('0.0.0.0', 80)[0][-1]
    s = socket.socket()
    s.bind(addr)
    s.listen(1)
    print('Servidor web iniciado en', addr)

    while True:
        cl, addr = s.accept()
        request = cl.recv(1024).decode()
        print('Solicitud de', addr, request)


        if '/on' in request:
            sistema_habilitado = True
            led_rojo_encendido = False
            led_rojo_1.off()
            led_rojo_2.off()
            estado_espera = False
            print("Sistema HABILITADO remotamente")
        elif '/off' in request:
            sistema_habilitado = False
            print("Sistema DESHABILITADO remotamente")


        try:
            sensor.measure()
            temp = sensor.temperature()
        except:
            temp = "--"

        estado = "HABILITADO" if sistema_habilitado else "DESHABILITADO"

        # Página HTML
        html = f"""<!DOCTYPE html>
<html>
<head>
    <meta charset="UTF-8">
    <title>Control de Temperatura</title>
    <style>
        body {{
            font-family: Arial, sans-serif;
            text-align: center;
            background-color: #f4f4f4;
            color: #333;
        }}
        h2 {{
            color: #4CAF50;
        }}
        button {{
            padding: 10px 20px;
            font-size: 16px;
            margin: 10px;
            background-color: #4CAF50;
            color: white;
            border: none;
            border-radius: 5px;
            cursor: pointer;
        }}
        button:hover {{
            background-color: #45a049;
        }}
        p {{
            font-size: 18px;
        }}
    </style>
</head>
<body>
    <h2>Control de Temperatura - Proyecto</h2>
    <p><strong>Temperatura actual:</strong> {temp}°C</p>
    <p><strong>Estado del sistema:</strong> {estado}</p>
    <form action="/on" method="get">
        <button>Activar</button>
    </form>
    <br>
    <form action="/off" method="get">
        <button>Desactivar</button>
    </form>
</body>
</html>"""

        cl.send("HTTP/1.1 200 OK\r\nContent-Type: text/html\r\n\r\n")
        cl.sendall(html.encode())
        cl.close()


def sonar_buzzer():
    for _ in range(2):
        for _ in range(3):
            buzzer.freq(1000)
            buzzer.duty_u16(32768)
            time.sleep(0.3)
            buzzer.duty_u16(0)
            time.sleep(0.7)


def main():
    global sistema_habilitado, led_rojo_encendido, tiempo_inicio_led_rojo, estado_espera
    connect_wifi()

    led_energia.value(1)
    led_verde_1.on()
    tiempo_inicio = 0

    while True:
        if estado_espera:

            time.sleep(1)
            continue

        if not sistema_habilitado:

            motor_in1.off()
            motor_in2.off()
            buzzer.deinit()
            led_amarillo.off()
            led_verde_2.off()
            time.sleep(1)
            continue

        try:
            sensor.measure()
            temp = sensor.temperature()
            print("Temperatura:", temp)
        except Exception as e:
            print("Error lectura sensor:", e)
            time.sleep(2)
            continue

        if temp > temp_umbral:
            motor_in1.on()
            motor_in2.off()
            led_amarillo.on()
            led_verde_2.off()

            if tiempo_inicio == 0:
                tiempo_inicio = time.time()

            if time.time() - tiempo_inicio > tiempo_max_regulacion:
                print("No se ha logrado regular la temperatura.")


                led_rojo_1.on()
                led_rojo_2.on()
                led_amarillo.off()


                motor_in1.off()
                motor_in2.off()


                sonar_buzzer()

                tiempo_inicio = 0
                led_rojo_encendido = True
                tiempo_inicio_led_rojo = time.time()
                estado_espera = True
        else:

            motor_in1.off()
            motor_in2.off()
            buzzer.deinit()
            led_amarillo.off()
            led_verde_2.on()
            tiempo_inicio = 0


        if led_rojo_encendido and (time.time() - tiempo_inicio_led_rojo >= tiempo_led_rojo):
            led_rojo_1.off()
            led_rojo_2.off()
            led_rojo_encendido = False

        time.sleep(1)


_thread.start_new_thread(web_server, ())
main()




